In [1]:
pip install --ignore-installed -r requirements.txt

  Using cached langchain-0.1.19-py3-none-any.whl.metadata (13 kB)
  Using cached langchain_community-0.0.38-py3-none-any.whl.metadata (8.7 kB)
  Using cached chromadb-0.5.0-py3-none-any.whl.metadata (7.3 kB)
  Using cached PyYAML-6.0.1-cp312-cp312-macosx_10_9_x86_64.whl.metadata (2.1 kB)
  Using cached SQLAlchemy-2.0.30-cp312-cp312-macosx_10_9_x86_64.whl.metadata (9.6 kB)
  Using cached aiohttp-3.9.5-cp312-cp312-macosx_10_9_x86_64.whl.metadata (7.5 kB)
  Using cached dataclasses_json-0.6.5-py3-none-any.whl.metadata (25 kB)
  Using cached langchain_core-0.1.52-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_text_splitters-0.0.1-py3-none-any.whl.metadata (2.0 kB)
  Using cached langsmith-0.1.56-py3-none-any.whl.metadata (13 kB)
  Using cached numpy-1.26.4-cp312-cp312-macosx_10_9_x86_64.whl.metadata (61 kB)
  Using cached pydantic-2.7.1-py3-none-any.whl.metadata (107 kB)
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached tenacity-8.3.0-py3-none-an

# Setup OpenAI credentials

In [9]:
import os
from getpass import getpass

OPENAI_API_KEY = getpass()
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

 ········


# Load project files into documents

In [62]:
from langchain.document_loaders import TextLoader
from langchain_community.document_loaders import DirectoryLoader

project_dir = "/Users/htrinter/workspace/Open-Multiple-URLs/Open-Multiple-URLs/src"
file_extensions = [".vue", ".ts"]

project_dir = "/Users/htrinter/workspace/zenfood-app/lib"
file_extensions = [".dart"]

docs = []
for extension in file_extensions:
    loader = DirectoryLoader(project_dir, glob=f"**/*{extension}", loader_cls=TextLoader, recursive=True)
    docs.extend(loader.load())

print(f"loaded {len(docs)} docs")

loaded 50 docs


# Split documents into chunks

In [64]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(docs)

# Generate embeddings from chunks

In [65]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

# Ask a question about your code

In [67]:
from langchain_openai import OpenAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

llm = OpenAI()

system_prompt = (
    "Use the given context to answer the question."
    "If you don't know the answer, say you don't know."
    "Context: {context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(vectorstore.as_retriever(), question_answer_chain)

q = "What does the flag isStoredValuesLoaded do?"
q = "I want to add an option to load the links in a new window. What do I have to do to implement this?"
q = "How do I add a chart to the montly screen?"
chain.invoke({"input": q})

{'input': 'How do I add a chart to the montly screen?',
 'context': [Document(page_content='final List<List<Widget>> actionOptions = [\n      [\n        Padding(\n          padding: const EdgeInsets.only(right: 8.0),\n          child: IconButton(\n              icon: const Icon(Icons.chevron_right_rounded),\n              onPressed: () => journalController.incrementDate()),\n        )\n      ],\n      [],\n      []\n    ];', metadata={'source': '/Users/htrinter/workspace/zenfood-app/lib/conversation/presentation/conversation_screen_widget.dart'}),
  Document(page_content='final List<Widget> titleOptions = [\n      MouseRegion(\n          cursor: SystemMouseCursors.click,\n          child: GestureDetector(\n              onTap: () {\n                journalController.setDateNow();\n              },\n              child: Text(journalState.valueOrNull?.displayDateStr ?? ""))),\n      const Text("Coach"),\n      const Text("Werkzeuge")\n    ];', metadata={'source': '/Users/htrinter/workspa